In [ ]:
# Import Libraries
import warnings
warnings.filterwarnings('ignore')
import time
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
# 모델의 reproducibility를 위해 학습 환경 및 random seed 고정
import numpy as np
import tensorflow as tf
import random

seed_num = 256
np.random.seed(seed_num)
random.seed(seed_num)
tf.random.set_seed(seed_num)

from keras import backend as K

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads = 1,
                                        inter_op_parallelism_threads = 1)
sess = tf.compat.v1.Session(graph = tf.compat.v1.get_default_graph(), config = session_conf)
K.set_session(sess)

In [ ]:
#모델 라이브러리 가져오기 
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
import tensorflow as tf
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
x_train = np.load('/content/drive/Shareddrives/2022-1 KUBIG 음악추천시스템/melspectogram.npy')

In [ ]:
target = []
for i in range(200):
  target.append(0)
for i in range(200):
  target.append(1)
for i in range(200):
  target.append(2)
for i in range(200):
  target.append(3)
for i in range(200):
  target.append(4)

In [ ]:
y_train = np.array(target)

In [ ]:
import cv2
new = []
for i in range(1000):
  new.append(cv2.resize(x_train[i], (256, 256), interpolation = cv2.INTER_LANCZOS4))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, random_state = 256, test_size = 0.2)

In [ ]:
import numpy as np

from sklearn.metrics import accuracy_score
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.utils.np_utils import to_categorical

# parameters for data load
num_words = 30000
maxlen = 400
test_split = 0.3


# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)

# y_train = y_data[:1395]
# y_test = y_data[1395:]

# 데이터의 모양 출력하기
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(800, 128, 2584)
(200, 128, 2584)
(800,)
(200,)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

# 기본 RNN 모델을 구현하기 위한 함수
def stacked_vanilla_rnn():
    model = Sequential()
    model.add(SimpleRNN(400, input_shape = (128, 2584), return_sequences = True))   # return_sequences parameter has to be set True to stack
    model.add(SimpleRNN(400, return_sequences = False))
    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    adam = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [ ]:
model = KerasClassifier(build_fn = stacked_vanilla_rnn, epochs = 100, batch_size = 64, verbose = 1)

In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_pred, y_test))

Epoch 1/100
13/13 [==============================] - 7s 263ms/step - loss: 2.1007 - accuracy: 0.2138
Epoch 2/100
13/13 [==============================] - 3s 257ms/step - loss: 1.6772 - accuracy: 0.1838
Epoch 3/100
13/13 [==============================] - 4s 268ms/step - loss: 1.6321 - accuracy: 0.1863
Epoch 4/100
13/13 [==============================] - 4s 272ms/step - loss: 1.6197 - accuracy: 0.1900
Epoch 5/100
13/13 [==============================] - 3s 260ms/step - loss: 1.6175 - accuracy: 0.2025
Epoch 6/100
13/13 [==============================] - 3s 262ms/step - loss: 1.6341 - accuracy: 0.1925
Epoch 7/100
13/13 [==============================] - 3s 264ms/step - loss: 1.6298 - accuracy: 0.1975
Epoch 8/100
13/13 [==============================] - 3s 256ms/step - loss: 1.6340 - accuracy: 0.2013
Epoch 9/100
13/13 [==============================] - 3s 262ms/step - loss: 1.6406 - accuracy: 0.1937
Epoch 10/100
13/13 [==============================] - 3s 265ms/step - loss: 1.6525 - accura

In [ ]:
# 단층 LSTM을 구현하기 위한 함수
from keras.layers import LSTM

def lstm():
    model = Sequential()
    model.add(LSTM(400, input_shape = (128, 2584), return_sequences = False))
    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    adam = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [ ]:
model = KerasClassifier(build_fn = lstm, epochs = 100, batch_size = 64, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_pred, y_test))

Epoch 1/100
13/13 [==============================] - 5s 97ms/step - loss: 1.6550 - accuracy: 0.1912
Epoch 2/100
13/13 [==============================] - 1s 92ms/step - loss: 1.6140 - accuracy: 0.2075
Epoch 3/100
13/13 [==============================] - 1s 93ms/step - loss: 1.6117 - accuracy: 0.1950
Epoch 4/100
13/13 [==============================] - 1s 93ms/step - loss: 1.6110 - accuracy: 0.1937
Epoch 5/100
13/13 [==============================] - 1s 93ms/step - loss: 1.6139 - accuracy: 0.2075
Epoch 6/100
13/13 [==============================] - 1s 93ms/step - loss: 1.6127 - accuracy: 0.1825
Epoch 7/100
13/13 [==============================] - 1s 93ms/step - loss: 1.6165 - accuracy: 0.2062
Epoch 8/100
13/13 [==============================] - 1s 95ms/step - loss: 1.6202 - accuracy: 0.1937
Epoch 9/100
13/13 [==============================] - 1s 107ms/step - loss: 1.6149 - accuracy: 0.1937
Epoch 10/100
13/13 [==============================] - 2s 117ms/step - loss: 1.6124 - accuracy: 0.20

In [ ]:
# 다층 LSTM을 구현하기 위한 함수
def stacked_lstm():
    model = Sequential()
    model.add(LSTM(400, input_shape = (128, 2584), return_sequences = True))
    model.add(LSTM(400, return_sequences = False))
    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    adam = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [ ]:
model = KerasClassifier(build_fn = stacked_lstm, epochs = 100, batch_size = 64, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_pred, y_test))

Epoch 1/100
13/13 [==============================] - 4s 122ms/step - loss: 1.6716 - accuracy: 0.2037
Epoch 2/100
13/13 [==============================] - 2s 118ms/step - loss: 1.6276 - accuracy: 0.2075
Epoch 3/100
13/13 [==============================] - 2s 118ms/step - loss: 1.6231 - accuracy: 0.1912
Epoch 4/100
13/13 [==============================] - 2s 118ms/step - loss: 1.6162 - accuracy: 0.1912
Epoch 5/100
13/13 [==============================] - 2s 118ms/step - loss: 1.6161 - accuracy: 0.1937
Epoch 6/100
13/13 [==============================] - 2s 117ms/step - loss: 1.6260 - accuracy: 0.1637
Epoch 7/100
13/13 [==============================] - 2s 118ms/step - loss: 1.6221 - accuracy: 0.1975
Epoch 8/100
13/13 [==============================] - 2s 118ms/step - loss: 1.6201 - accuracy: 0.1963
Epoch 9/100
13/13 [==============================] - 2s 118ms/step - loss: 1.6263 - accuracy: 0.1912
Epoch 10/100
13/13 [==============================] - 2s 118ms/step - loss: 1.6121 - accura

In [ ]:
# 단층 GRU를 구현하기 위한 함수
from keras.layers import GRU

def gru():
    model = Sequential()
    model.add(GRU(400, input_shape = (128, 2584), return_sequences = False))
    model.add(Dense(5))
    model.add(Activation('softmax'))
    
    adam = tf.keras.optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [ ]:
model = KerasClassifier(build_fn = gru, epochs = 100, batch_size = 64, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_pred, y_test))

Epoch 1/100
13/13 [==============================] - 3s 81ms/step - loss: 1.8478 - accuracy: 0.1850
Epoch 2/100
13/13 [==============================] - 1s 77ms/step - loss: 1.6583 - accuracy: 0.1963
Epoch 3/100
13/13 [==============================] - 1s 76ms/step - loss: 1.6261 - accuracy: 0.2100
Epoch 4/100
13/13 [==============================] - 1s 76ms/step - loss: 1.6156 - accuracy: 0.1838
Epoch 5/100
13/13 [==============================] - 1s 78ms/step - loss: 1.6128 - accuracy: 0.2125
Epoch 6/100
13/13 [==============================] - 1s 78ms/step - loss: 1.6259 - accuracy: 0.1550
Epoch 7/100
13/13 [==============================] - 1s 77ms/step - loss: 1.6207 - accuracy: 0.2037
Epoch 8/100
13/13 [==============================] - 1s 78ms/step - loss: 1.6244 - accuracy: 0.1875
Epoch 9/100
13/13 [==============================] - 1s 78ms/step - loss: 1.6330 - accuracy: 0.1762
Epoch 10/100
13/13 [==============================] - 1s 78ms/step - loss: 1.6212 - accuracy: 0.1513